# Model 4
## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [3]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_5 = pd.read_csv("data/informatikkurse.csv")
data_model_5 = data_model_5[data_model_5['model_2'] == 0].copy()
data_model_5

C:\Users\mhu\AppData\Local\Temp\ipykernel_29476\616276209.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_5 = pd.read_csv("data/informatikkurse.csv")


,Unnamed: 0,veranstaltung_id,veranstaltung_titel,kursbeschreibung,kursformat_original,sprache_original,lernziele,lernmethode,literatur,voraussetzungen,...,medizin_info,wirt_info,recht_info,geo_info,bau_info,base_info,model_1_b,model_2,model_3,model_4
430,431,2879786,Projekt 1: Verkehr,NaN,Seminar/ Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
713,714,1798130,Biometrie und Epidemiologie,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,1,0,1,0
714,715,1798134,Tutorial: Biometrie,Häufig benutzte methodische Ansätze sowie der ...,Tutorium,NaN,NaN,NaN,"Weiß, C. (1999): Basiswissen Medizinische Stat...","Kenntnisse aus dem Stoffgebiet, die in der Vor...",...,1,0,0,0,0,0,1,0,0,0
715,716,1798138,Tutorial: Biometrie,Häufig benutzte methodische Ansätze sowie der ...,Tutorium,NaN,NaN,NaN,"Weiß, C. (1999): Basiswissen Medizinische Stat...","Kenntnisse aus dem Stoffgebiet, die in der Vor...",...,1,0,0,0,0,0,1,0,0,0
716,717,1798139,Statistisches Seminar für Fortgeschrittene,NaN,Seminar,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50423,50424,1592323,Quantitative Methoden - Statistik und Operatio...,Statistik ; Methoden der deskriptiven Statisti...,Gruppenübung,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
50424,50425,1592324,Projektseminar IWI,Die Inhalte differeren je nach Anbieter und Th...,Projektseminar,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
50425,50426,1592325,Projektseminar IWI,Die Inhalte differeren je nach Anbieter und Th...,Projektseminar,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
50426,50427,1592326,Simulationsmethoden,Die Simulation ist eine relevante Forschungsme...,Integrierte Vorlesung,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [4]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [5]:
# Anzeige der Spaltennamen von data_model_5
print(data_model_5.columns.tolist())

data_model_5 = data_model_5[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_5.head()

data_model_5 = data_model_5.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_5['titel_kursbesch'] = data_model_5['veranstaltung_titel'] + ' ' + data_model_5['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_5['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [6]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

# Model

Wir definieren unser Model.

In [7]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [8]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model_5 = BERTopic(
  embedding_model=embedding_model,
  # min_topic_size=100,
  nr_topics="auto", 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 20,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [9]:
topic_model_5.fit(docs)
topic_model_5.get_topic_info()

2026-02-05 16:07:56,841 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 266/266 [00:05<00:00, 50.21it/s] 
2026-02-05 16:08:02,244 - BERTopic - Embedding - Completed ✓
2026-02-05 16:08:02,245 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 16:08:21,800 - BERTopic - Dimensionality - Completed ✓
2026-02-05 16:08:21,801 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 16:08:22,483 - BERTopic - Cluster - Completed ✓
2026-02-05 16:08:22,484 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 16:08:22,866 - BERTopic - Representation - Completed ✓
2026-02-05 16:08:22,867 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 16:08:22,872 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 16:08:23,512 - BERTopic - Representation - Completed ✓
2026-02-05 16:08:23,515 - BERTopic - Topic reduction - R

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5952,-1_data_management_systems_wirtschaftsinformatik,"[data, management, systems, wirtschaftsinforma...",[Advanced Analytics and Applications Advanced ...
1,0,1104,0_wirtschaftsinformatik_systems_learning_digital,"[wirtschaftsinformatik, systems, learning, dig...",[Bachelorseminar Information Management (Prof....
2,1,689,1_informatik_methoden_systems_design,"[informatik, methoden, systems, design, system...",[Einführung in die Bauinformatik und Programmi...
3,2,196,2_wirtschaftsinformatik_wirtschaftsinformatik ...,"[wirtschaftsinformatik, wirtschaftsinformatik ...",[ISDL-eFin-B: Electronic Finance E-Finance bez...
4,3,166,3_epidemiologie medizinische biometrie_biometr...,"[epidemiologie medizinische biometrie, biometr...","[Querschnittsbereich Epidemiologie, medizinisc..."
5,4,145,4_geoinformatik_geoinformationssysteme_geoinfo...,"[geoinformatik, geoinformationssysteme, geoinf...",[Geo Web Services Students attending this cour...
6,5,128,5_e business_e commerce_environmental engineer...,"[e business, e commerce, environmental enginee...","[Übung: Grundlagen des E-Business (Übung) , IT..."
7,6,128,6_studentisches_dient klärung individueller_kl...,"[studentisches, dient klärung individueller, k...",[Interdisciplinary Business Seminar Content an...


## Topic Visualiszieren, Outlier Reduzieren, Topics mergen

In [ ]:
data_model_5.visualize_topics()
#data_model_5.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# # Outlier reduzieren
# # BERTopic auf Test-Daten anwenden
# topics, probs = data_model_5.transform(docs)
# print(data_model_5.get_topic_freq())

# # # Outlier reduzieren
# topics = data_model_5.reduce_outliers(docs, topics)
# data_model_5.update_topics(docs, topics=topics)

In [ ]:
data_model_5.get_topic_info()

## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [ ]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
data_model_5.update_topics(docs, representation_model=rep)
data_model_5.get_topic_info() 

100%|██████████| 13/13 [02:22<00:00, 10.94s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2024,-1_Human-Computer Interaction,[Human-Computer Interaction],[Praktikum DBMS: Ubiquitous Texttechnologies I...
1,0,624,0_Scientific Work in Informatics,[Scientific Work in Informatics],"[Informatik-Kolloquium , Übung zu Informatik A..."
2,1,324,1_Software Engineering Project,[Software Engineering Project],"[Software-Projektseminar , Software-Praktikum ..."
3,2,316,2_Robotics and Deep Learning,[Robotics and Deep Learning],"[Machine Learning , Praktikum - Roboy Student ..."
4,3,261,3_Wirtschaftsinformatik und IT-Management,[Wirtschaftsinformatik und IT-Management],[WInf-InfWiMa: Informations- und Wissensmanage...
5,4,224,4_Economics of Security and Privacy,[Economics of Security and Privacy],[Bachelor/Master-Seminar - Usable Security and...
6,5,205,5_Efficient Algorithms Design,[Efficient Algorithms Design],[Effiziente Algorithmen 1 Ein zentrales Proble...
7,6,205,6_Text Imaging Technologies,[Text Imaging Technologies],[VCP-PR Praktikum Visual Computing Projekt-Pra...
8,7,197,7_Fuzzy methods in education,[Fuzzy methods in education],[Praktische Übung zu: Fuzzy-Methoden Kurzfassu...
9,8,183,8_Grundlagen der Hardwareinformatik,[Grundlagen der Hardwareinformatik],[Systemprogrammierung In diesem Praktikum werd...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")